In [4]:
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from math import inf
import numpy as np
from matplotlib import pyplot as plt
import math
import sys

def my_ISOMAP(X, k, n_neighbors=5, radius = 1.0, method = 'kNN', standardize=False):
    """
    Compute the Isomap embedding of dataset X

    Parameters:
    -----------
    X: dataframe
        dataframe with n observation and p features (nxp)
    k: int
        number of principal components we want to project on
    n_neighbors: int, default=5
        number of neighbors to take into account for each point to build the graph of geodesic distance
        Used only if 'method = kNN'
    radius: float, default=1.0
        radius to use inside sklearn.neighbors.NearestNeighbors algorithm
        Used only if 'method = balls'
    method: ['kNN', 'balls']
        if kNN: find the n_neighbors nearest-neighbor(s) of a point.
        if balls: find the n_neighbors nearest-neighbor(s) within a given radius of a point

    Returns:
    --------
    numpy array of shape (n, k)
        Return the projected data (nxk), where k is the number of principal components we have projected on
        The low-dimensional representation of the dataset X

    The algorithm procedes as follows:
        - for each point, select the n_neighbors nearest neighbor(s)
        - compute the graph (geodesic) distance using: graph[i,j] == 1 iff point i is one of the n_neighbors nearest neighbor(s) of point j
        - apply floyd-warshall algorithm to the previous graph
        - apply double centering on the adiaceny matrix of the graph obtained before
        - compute the eigenvalues/eigenvectors decomposition
        - choose the first k eigenvector(s)
    """
    if standardize==True:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        
    if method=='kNN':
        # K nearest neighbors
        neigh = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(X)

        # Construct neighborhood graph
        graph = neigh.kneighbors_graph(X).toarray()

    elif method=='balls':
        # K nearest neighbors
        neigh = NearestNeighbors(algorithm='ball_tree', radius = radius).fit(X)

        # Construct neighborhood graph
        graph = neigh.radius_neighbors_graph(X).toarray()

    else:
        raise RuntimeError('error: enter a valid method. Valid methods are [\'kNN\', \'balls\']')
    
    np.set_printoptions(threshold=sys.maxsize)
    print('before', graph)

    # apply floyd-warshall
    n_nodes = graph.shape[0]

    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j and graph[i,j]== 0:
                graph[i,j] = inf # use a large number instead of math.inf for numerical reason

    dist = np.copy(graph)
    for k in range(n_nodes):
        for i in range(n_nodes):
            for j in range(n_nodes):
                if dist[i,k] + dist[k,j] < dist[i,j]:
                    dist[i,j] = dist[i,k] + dist[k,j]

    graph = dist

    """
    for k in range(n_nodes):
        for i in range(n_nodes):
            for j in range(n_nodes):
                if graph[i,j] > graph[i,k] + graph[k,j]:
                    graph[i,j] = graph[i,k] + graph[k,j]
    """


    # apply double centering on graph adiacency matrix
    J = np.eye(n_nodes) - 1/n_nodes * np.dot(np.ones(n_nodes), np.ones(n_nodes).T)

    G = -1/2 * np.dot(J, np.dot(graph, J))


    # eigenvector decomposition
    eig_vals, eig_vecs = np.linalg.eigh(G) # use .eigh to have the real-value decomposition. Since 'graph' is not guaranteed to be symmetric, B could be complex.

    ###
    print(G)
    print(np.dot(eig_vecs, np.dot(np.diag(eig_vals), np.linalg.inv(eig_vecs))))
    ###
    sorted_indices = np.argsort(eig_vals)[::-1]
    sorted_eigvecs = eig_vecs[:,sorted_indices]

    # select the first k eigenvector(s) and project
    top_k_eigvecs = sorted_eigvecs[:k]

    projection_matrix = top_k_eigvecs.T

    return np.dot(G, projection_matrix)